In [1]:
from qiskit import QuantumCircuit, execute, Aer

In [2]:
class QuantumLeftShift:
    def __init__(self, data_qubits, seed, circular = False):
        self.seed_string = seed
        
        #to follow qiskit's little endian format
        self.seed = list(map(int, list(seed)))[::-1]
        self.n = data_qubits
        
        if len(seed) != self.n:
            raise ValueError("length of seed should be equal to number of data qubits")
    
        self.qc = QuantumCircuit(self.n + self.n + 1, self.n)
        self.shifts_done = 0
        self.circular = circular
        self.__prepareCircuit()
        
        self.shiftGate = self.constructShiftGate()
        
    def __repr__(self):
        return "QuantumShiftLeft(data_qubits = %d, seed = \"%s\", circular = %s)" % (
                                                                                        self.n,
                                                                                        self.seed_string,
                                                                                        str(self.circular)
                                                                                    )
    
    def __str__(self):
        print(self.qc)
        return "Data qubits: %d\nInitial seed: %s\nShifts performed: %d\nCircular: %s" % (
                                                                                              self.n, 
                                                                                              self.seed_string, 
                                                                                              self.shifts_done,
                                                                                              str(self.circular)  
                                                                                        )
    
    
    def __prepareCircuit(self):
        for i in range(len(self.seed)):
            if self.seed[i] == 1:
                self.qc.x(i)
        if(self.circular):
            self.qc.x(self.n + self.n)
        self.qc.barrier(range(self.n + self.n + 1))            
    
    def constructShiftGate(self):
        if self.circular:
            name = "Circular Left Shift"
        else:
            name = "Left Shift"
            
        shiftCirc = QuantumCircuit(self.n + self.n + 1, name = name)
        for qubit in range(1, self.n + self.n)[::-1]:
            shiftCirc.swap(qubit, qubit - 1)
        
        shiftCirc.cx(0, self.n)
        shiftCirc.toffoli(self.n + self.n, self.n, 0)
        shiftCirc.cx(0, self.n)
        
        shiftGate = shiftCirc.to_gate()
        return shiftGate
    
    def shift(self):
        self.shifts_done += 1
        self.qc.append(self.shiftGate, range(self.n + self.n + 1))
    
    def getRegisterState(self):
        self.qc.barrier(range(self.n + self.n + 1)) 
        self.qc.measure(range(self.n), range(self.n))
        self.qc.barrier(range(self.n + self.n + 1))
        
        backend = Aer.get_backend('statevector_simulator')
        registerState = execute(self.qc, backend, shots = 1024).result().get_counts()
        
        return registerState        

In [3]:
qsl = QuantumLeftShift(4, "1011", circular = True)

In [4]:
qsl.shift()
qsl.shift()
qsl.getRegisterState()

{'1110': 8}

In [5]:
qsl

QuantumShiftLeft(data_qubits = 4, seed = "1011", circular = True)

In [6]:
print(qsl)

     ┌───┐ ░ ┌──────────────────────┐┌──────────────────────┐ ░ ┌─┐          ░ 
q_0: ┤ X ├─░─┤0                     ├┤0                     ├─░─┤M├──────────░─
     ├───┤ ░ │                      ││                      │ ░ └╥┘┌─┐       ░ 
q_1: ┤ X ├─░─┤1                     ├┤1                     ├─░──╫─┤M├───────░─
     └───┘ ░ │                      ││                      │ ░  ║ └╥┘┌─┐    ░ 
q_2: ──────░─┤2                     ├┤2                     ├─░──╫──╫─┤M├────░─
     ┌───┐ ░ │                      ││                      │ ░  ║  ║ └╥┘┌─┐ ░ 
q_3: ┤ X ├─░─┤3                     ├┤3                     ├─░──╫──╫──╫─┤M├─░─
     └───┘ ░ │                      ││                      │ ░  ║  ║  ║ └╥┘ ░ 
q_4: ──────░─┤4 Circular Left Shift ├┤4 Circular Left Shift ├─░──╫──╫──╫──╫──░─
           ░ │                      ││                      │ ░  ║  ║  ║  ║  ░ 
q_5: ──────░─┤5                     ├┤5                     ├─░──╫──╫──╫──╫──░─
           ░ │                      ││  